In [10]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
import statsmodels.api as sm

DATA_DIR = "/Users/stutipatel/Desktop/F1-pitstop-strategy-optimization/F1-Pit-Stop-Strategy-Recommender/data"

races = pd.read_csv(os.path.join(DATA_DIR, "races.csv"))
circuits = pd.read_csv(os.path.join(DATA_DIR, "circuits.csv"))
results = pd.read_csv(os.path.join(DATA_DIR, "results.csv"))
pits = pd.read_csv(os.path.join(DATA_DIR, "pit_stops.csv"))
laps = pd.read_csv(os.path.join(DATA_DIR, "lap_times.csv"))

print("races:", races.shape)
print("circuits:", circuits.shape)
print("results:", results.shape)
print("pit_stops:", pits.shape)
print("lap_times:", laps.shape)

races.head()

races: (1125, 18)
circuits: (77, 9)
results: (26759, 18)
pit_stops: (11371, 7)
lap_times: (589081, 6)


,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N


In [11]:
# cell 3 - parsing dates and numeric conversions

# parse dates in races if not already parsed
if 'date' in races.columns:
    races['date'] = pd.to_datetime(races['date'], errors='coerce')

# ensure milliseconds -> seconds in lap_times
if 'milliseconds' in laps.columns:
    laps['lap_time_s'] = laps['milliseconds'] / 1000.0
else:
    # If lap times already in some column, adapt
    pass

# pit stop durations: some CSVs have 'duration' as string like '23.456'
if 'duration' in pits.columns:
    # convert to float where possible:
    def safe_float(x):
        try:
            return float(x)
        except:
            return np.nan
    pits['duration_s'] = pits['duration'].apply(safe_float)
else:
    pits['duration_s'] = np.nan

# numeric corrections in results
for col in ['position', 'grid']:
    if col in results.columns:
        results[col] = pd.to_numeric(results[col], errors='coerce')

# show converted columns
display(laps[['raceId','driverId','lap','lap_time_s']].head())
display(pits[['raceId','driverId','lap','stop','duration_s']].head())

,raceId,driverId,lap,lap_time_s
0,841,20,1,98.109
1,841,20,2,93.006
2,841,20,3,92.713
3,841,20,4,92.803
4,841,20,5,92.342


,raceId,driverId,lap,stop,duration_s
0,841,153,1,1,26.898
1,841,30,1,1,25.021
2,841,17,11,1,23.426
3,841,4,12,1,23.251
4,841,13,13,1,23.842


In [12]:
# cell 4 - cleaning basics

# drop exact duplicate rows across the loaded tables (if any)
races.drop_duplicates(inplace=True)
circuits.drop_duplicates(inplace=True)
results.drop_duplicates(inplace=True)
pits.drop_duplicates(inplace=True)
laps.drop_duplicates(inplace=True)

# Inspect missingness summary for core files
def missing_summary(df, name):
    print(f"--- {name} ---")
    print(df.isna().sum().sort_values(ascending=False).head(10))
    print(f"Total rows: {len(df)}\n")

missing_summary(races, "races")
missing_summary(results, "results")
missing_summary(pits, "pit_stops")
missing_summary(laps, "lap_times")

# remove rows with obviously invalid IDs or lap times (if any)
laps = laps[laps['lap'] > 0]   # remove nonpositive laps
laps = laps[laps['lap_time_s'].notna()]

# For pit stops, ensure lap>0
pits = pits[pits['lap'] > 0]


--- races ---
raceId         0
year           0
sprint_date    0
quali_time     0
quali_date     0
fp3_time       0
fp3_date       0
fp2_time       0
fp2_date       0
fp1_time       0
dtype: int64
Total rows: 1125

--- results ---
position           10953
resultId               0
laps                   0
fastestLapSpeed        0
fastestLapTime         0
rank                   0
fastestLap             0
milliseconds           0
time                   0
points                 0
dtype: int64
Total rows: 26759

--- pit_stops ---
duration_s      517
raceId            0
driverId          0
stop              0
lap               0
time              0
duration          0
milliseconds      0
dtype: int64
Total rows: 11371

--- lap_times ---
raceId          0
driverId        0
lap             0
position        0
time            0
milliseconds    0
lap_time_s      0
dtype: int64
Total rows: 589081



In [13]:
status = pd.read_csv(os.path.join(DATA_DIR, "status.csv"))
results = results.merge(status, on="statusId", how="left")

# Check examples of statuses
results['status'].value_counts().head(10)

status
Finished           7674
+1 Lap             4037
Engine             2026
+2 Laps            1613
Accident           1062
Did not qualify    1025
Collision           854
Gearbox             810
Spun off            795
+3 Laps             731
Name: count, dtype: int64

In [14]:
# Replace missing position values with a custom label for DNF
results['position'] = results['position'].fillna(-1)
results['positionLabel'] = results['position'].apply(lambda x: 'DNF' if x == -1 else 'Finished')